In [5]:
import numpy as np
from numpy import random
import random
import matplotlib.pyplot as plt
from scipy.signal import cwt,ricker
from scipy import stats
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import os
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.optimizers
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.models import load_model
from tensorflow.keras.layers import Flatten,concatenate,Layer,Dense,LSTM,Activation,MaxPooling2D,Dropout,Conv2D,BatchNormalization,Reshape,UpSampling2D,ZeroPadding2D
#import nibabel as nib
#import pydicom as dicom
#from pydicom.pixel_data_handlers.util import apply_voi_lut
#import tensorflow_io as tfio
import radiomics
from radiomics.featureextractor import RadiomicsFeatureExtractor
import SimpleITK as sitk
import pandas as pd


2022-08-04 18:59:22.520699: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Example: Load An Image

In [100]:
# the folder name by label
class_names = ['pos','neg']

# the image dictionary we are loading
dict = './covid19/kcv/'

In [101]:
# load one image with their label to memory
def load_image( imageName ):

    for i in class_names:
        path = os.path.join(dict, i)
        label_num = class_names.index(i)
        try:
            img = sitk.ReadImage(dict + i +'/' + imageName)
            trainingdata= [sitk.GetArrayFromImage(img).reshape(512,512,1),label_num]
        except:
            pass
            
    return trainingdata
    

In [102]:
singleImage = load_image('COVID-19_0414.png')

In [103]:
# get the label and the image
image = singleImage[0]
label = singleImage[1]

In [104]:
print(label)

0


# Example: Extract the Features From a Image

In [79]:
# the image dictionary and the image name want to extract feature from
dict = './covid19/kcv/pos/'

def ExtractFeaturesFromImages(imageName):

    o1=[]
    o2=[]
    o3=[]
    o4=[]
    o5=[]

    extractor = RadiomicsFeatureExtractor()
    # Disable all classes except firstorder
    extractor.disableAllFeatures()
    extractor.enableFeaturesByName(firstorder=['Skewness'])  # change here to extract
    extractor.enableFeaturesByName(glcm=['Autocorrelation'])
    extractor.enableFeaturesByName(glrlm=['GrayLevelVariance','RunLengthNonUniformity'])
    extractor.enableFeaturesByName(glszm=['SizeZoneNonUniformity'])

    im = sitk.ReadImage(dict + imageName)

    ma_arr = np.ones(im.GetSize()[::-1])  # reverse the order as image is xyz, array is zyx
    ma = sitk.GetImageFromArray(ma_arr)
    ma.CopyInformation(im)  # Copy geometric info

    # extract features and fill the output array
    features = extractor.execute(im, ma)

    #output_array.append(features['original_ngtdm_Strength'])  # change here to extract
    o1.append(features['original_firstorder_Skewness'])
    o2.append(features['original_glcm_Autocorrelation'])
    o3.append(features['original_glrlm_GrayLevelVariance'])
    o4.append(features['original_glrlm_RunLengthNonUniformity'])
    o5.append(features['original_glszm_SizeZoneNonUniformity'])

            
    return o1,o2,o3,o4,o5


In [80]:
f1, f2, f3,f4,f5 = ExtractFeaturesFromImages('COVID-19_0414.png')

In [81]:
features = np.stack((f1,f2,f3,f4, f5),axis=1)

In [94]:
features

array([[-1.32527237e-01,  3.81368113e+01,  7.97970356e+00,
         4.01856025e+04,  7.46722482e+03]])

# Example: Load A Fin and Apply in a Image

In [95]:
image = np.array(image)

In [96]:
print(image.shape)

(512, 512, 1)


In [97]:
fin = load_model('finetune_Autocorrelation.h5')

In [87]:
inputimage = image.reshape(1,512,512,1)

In [99]:
fin.predict(inputimage)

1/1 [==============================] - 0s 30ms/step


array([[42.890038]], dtype=float32)

In [18]:
#NON 040

# Example Use the FINs in another Network

In [ ]:
# Load two Fins
fin1 = load_model('finetune_Skewness.h5')
fin2 = load_model('finetune_Autocorrelation.h5')


def MakeModel(modelname):
    input_layer = keras.Input(shape=(512,512,1))
    
    # load models and concat them
    l1 = fin1(input_layer)
    l2 = fin2(input_layer)
    
    n_d = concatenate([l1,l2])

    x = Dense(128, activation='relu',name='x1')(n_d)

    out = Dense(2, activation = 'softmax', name= modelname)(x)

    model = keras.Model(input_layer, out, name= modelname)
    return model